# 1. Test Random ENV with OpenAi Gym

In [1]:
import gym
import random
###################
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.optimizers import Adam
###################
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
###################
import warnings
warnings.filterwarnings("ignore")

2022-10-21 19:10:39.056367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 19:10:39.110187: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
env=gym.make("CartPole-v0")
states=env.observation_space.shape[0]
actions=env.action_space.n

In [3]:
episodes=10
for episode in range(1,episodes+1):
    state=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action=env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:35.0
Episode:2 Score:23.0
Episode:3 Score:11.0
Episode:4 Score:19.0
Episode:5 Score:15.0
Episode:6 Score:16.0
Episode:7 Score:19.0
Episode:8 Score:16.0
Episode:9 Score:36.0
Episode:10 Score:14.0


# 2. Create Deep Learning Model with Keras

In [4]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [5]:
model=build_model(states,actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [6]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

## Train DQN model

In [7]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4), metrics=['mae'])
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

2022-10-21 19:10:43.597353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 19:10:43.601891: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 33s 3ms/step - reward: 1.0000
398 episodes - episode_reward: 25.078 [8.000, 143.000] - loss: 3.648 - mae: 7.278 - mean_q: 13.502

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 39s 4ms/step - reward: 1.0000
142 episodes - episode_reward: 70.430 [15.000, 170.000] - loss: 10.329 - mae: 17.265 - mean_q: 33.530

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 40s 4ms/step - reward: 1.0000
63 episodes - episode_reward: 156.603 [53.000, 200.000] - loss: 14.241 - mae: 24.492 - mean_q: 48.371

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 45s 4ms/step - reward: 1.0000
51 episodes - episode_reward: 198.294 [149.000, 200.000] - loss: 17.750 - mae: 32.070 - mean_q: 64.054

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: 1.0000
50 e

## Test

In [8]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

# 4. Reloading Agent from Memory

In [9]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [10]:
del model
del dqn
del env

In [11]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [12]:
#Reload
dqn.load_weights('dqn_weights.h5f')

In [13]:
#Test again
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
